In [1]:
import numpy as np
from nptyping import NDArray
import random
import numpy.linalg as LA
import matplotlib.pyplot as plt
from typing import List, Set, Dict, Tuple, Optional, Any, Callable
from PIL import Image
import os
import json

In [2]:
from utils import load_image_to_arr, display_rgb, SVD2D
import dim_reduction
from spectral_data import ImageSpectralData
dim_reduction.init_basis_cache()
import cityscapes_helper
from dataset_creation import (
    MultiScaleImageSampler, 
    ImageChunkSummarizerOptions, 
    MultiScaleImageEncoder, 
    MultiScaleImageDecoder, 
    ImageAtScale, 
    CityscapesDatasetFactory, 
    WholeImage,
    CityScapesDataset
)
from cityscapes_helper import format_semantic_composition

In [3]:
# Ben Desptop
#cityscapes_helper.set_visual_data_zip_path("C:\\Users\\Ben\\Downloads\\leftImg8bit_trainvaltest.zip")
#cityscapes_helper.set_semantic_data_zip_path("C:\\Users\\Ben\\Downloads\\gtFine_trainvaltest.zip")

#Ben Laptop (via WSL)
cityscapes_helper.set_visual_data_zip_path("/mnt/c/Users/benki/Downloads/leftImg8bit_trainvaltest.zip")
cityscapes_helper.set_semantic_data_zip_path("/mnt/c/Users/benki/Downloads/gtFine_trainvaltest.zip")

cityscapes_helper.initialize_tool()

In [4]:
dataset = CityScapesDataset("full_dataset_v1_30000")
dataset.load()
normalized_features = dataset.get_normalized_features()
print(dataset.labels.shape)
print(dataset.features.shape)

0.npy
1.npy
2.npy
3.npy
4.npy
5.npy
6.npy
(30000, 30)
(30000, 701)


/home/benthehuman/school/vision/UW-Vision-Segmentation/dataset_creation.py:581: RuntimeWarning: invalid value encountered in true_divide
  feature_01 = (features - feature_min) / feature_range


In [5]:
toy_feature_subset = [(3, 'center_loc_polar'), (3, 'mean')]
toy_feature_mask = dataset.get_feature_subset_mask(toy_feature_subset)
tiny_features = dataset.features[:, toy_feature_mask]
tiny_features_normalized = dataset.get_normalized_features()[:, toy_feature_mask]
tiny_features_normalized.shape

(30000, 5)

In [15]:
foliage_index = 21
foliage_composition = dataset.labels[:, foliage_index]
majority_foliage_mask = foliage_composition > 0.5
print(majority_foliage_mask.sum()) # 4699 of 30000 features are mostly foliage.. sounds a bit high but whatever.  

4699


In [16]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [17]:
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel, random_state=0).fit(tiny_features_normalized, majority_foliage_mask)

In [ ]:
gpc.score(tiny_features_normalized, majority_foliage_mask)

Error: Session cannot generate requests

In [21]:
def anchor_grid(largest_scale: int, smallest_scale: int) -> List[List[Tuple[int, int]]]:
    CITYSCAPES_Y = 1024
    CITYSCAPES_X = 2048

    #total_margin = largest_scale - smallest_scale
    num_rows = (CITYSCAPES_Y - largest_scale) // smallest_scale
    num_cols = (CITYSCAPES_X - largest_scale) // smallest_scale
    result = []
    for row_i in range(num_rows):
        curr_row = []
        anchor_y = row_i * smallest_scale
        for col_i in range(num_cols):
            anchor_x = col_i * smallest_scale
            curr_row.append([anchor_y, anchor_x])
        result.append(curr_row)
    return result

def multiscale_image_grid(anchor_grid: List[List[Tuple[int, int]]], sampler: MultiScaleImageSampler, img: NDArray[Any]) -> List[List[List[ImageAtScale]]]:
    #flat_anchors =  [item for sublist in anchor_grid for item in sublist]
    #flat_multiscale_images = [sampler.sample(img, anchor) for anchor in flat_anchors]
    i = 0
    result = []
    for anchor_row in anchor_grid:
        image_row = []
        for anchor in anchor_row:
            multiscaleImages = sampler.sample(img, anchor)
            multiscaleImages.insert(0, WholeImage(img))
            image_row.append(multiscaleImages)
        result.append(image_row)
    return result

def feature_grid(image: NDArray[Any], encoder: MultiScaleImageEncoder, scales: List[int]):
    sampler = MultiScaleImageSampler(scales)
    ag = anchor_grid(scales[0], scales[-1])
    image_grid = multiscale_image_grid(ag, sampler, image)
    nrows = len(image_grid)
    ncols = len(image_grid[0])
    print(nrows, ncols)
    print(image_grid[0][0])

    dumbo = [[scaleImage] for scaleImage in image_grid[0][0]]
    first_feature = encoder.encode(dumbo)
    feature_size = first_feature.shape[1]

    wumb = 0
    result = np.zeros((len(image_grid), len(image_grid[0]), feature_size))
    for row_i in range(nrows):
        for col_i in range(ncols):
            print(wumb, end=", ")
            wumb += 1
            multiscale_image = image_grid[row_i][col_i]
            dumbo = [[scaleImage] for scaleImage in multiscale_image]
            result[row_i, col_i, :] =  encoder.encode(dumbo)[0]

    return result
    

s = [240, 96, 32]

whole_img_options = ImageChunkSummarizerOptions()
whole_img_options.downsample_factor = 8
whole_img_options.n_pcm_coeffs = [80, 40, 30]

opt_240 = ImageChunkSummarizerOptions()
opt_240.downsample_factor = 3
opt_240.n_pcm_coeffs = [120, 40, 20]

opt_96 = ImageChunkSummarizerOptions()
opt_96.downsample_factor = 2
opt_96.n_pcm_coeffs = [120, 40, 20]

opt_32 = ImageChunkSummarizerOptions()
opt_32.downsample_factor = 1
opt_32.n_pcm_coeffs = [60, 20, 15]

chunk_summarizer_options = [whole_img_options, opt_240, opt_96, opt_32]

multiscaleEncoder = MultiScaleImageEncoder(chunk_summarizer_options)
print(multiscaleEncoder.n_scales)
aceneid, img = cityscapes_helper.loadRandomVisualInfo()

    

4


In [22]:

feature_grid(img, multiscaleEncoder, s)

24 56
[<dataset_creation.WholeImage object at 0x7fa91c82f280>, <dataset_creation.ImageAtScale object at 0x7fa91c82fb50>, <dataset_creation.ImageAtScale object at 0x7fa91c82fe20>, <dataset_creation.ImageAtScale object at 0x7fa91c82f8b0>]
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 

KeyboardInterrupt: 